# PhotoZ ML with random forest trees applied to DC2

- Sylvie Dagoret-Campagne
- creation date : March 8th 2022
- affiliation : IJCLab/IN2P3/CNRS
- from A. Newman, DESC DE School, 2016
- CCIN2P3 : Use python kernel **desc**

## Requirements from LSST Science book:


Photometric redshifts for LSST will be applied and calibrated over the redshift range $0 < z < 4$
for galaxies to $r  \simeq 27.5$. For the majority of science cases, such as weak lensing and BAO, a subset
of galaxies with $i < 25.3$ will be used. For this high S/N gold standard subset over the
redshift interval, $0 < z < 3$, the photometric redshift requirements are:

- The root-mean-square scatter in photometric redshifts, $ \sigma_z/(1+z)$, must be smaller than 0.05, with a goal of 0.02.
- The fraction of $3\sigma $  outliers at all redshifts must be below 10%.
- The bias in $e_z = (z_{photo}−z_{spec})/(1+z_{spec})$ must be below 0.003 (or 0.01 for combined,analyses of weak lensing and baryon acoustic oscillations); the uncertainty in  z/(1+z) must also be known to similar accuracy.

In [ ]:
import os
import tables as tb
import pandas as pd
import numpy as np

In [ ]:
%matplotlib inline
from __future__ import division #avoids integer division; 1/2 = 0.5, not 0 
import numpy as np #numpy routines
from astropy.table import Table   #astropy routine for reading tables
import matplotlib.pyplot as plt   #plotting routines

# Random forest routine from scikit-learn:
from sklearn.ensemble import RandomForestRegressor

# Cross-Validation routines:
#from sklearn.cross_validation import KFold
from sklearn.model_selection import KFold
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import cross_val_predict
from sklearn.model_selection import cross_val_predict

# Input data

In [ ]:
#read in catalog of magnitudes and redshifts
#cat = Table.read('data_trim.fits.gz')

# How big is the catalog?
#print('Full catalog: ',len(cat),' objects')

# What information does it contain for each object?
#print(cat.colnames)
#print(cat)

In [ ]:
inputdatadir                   = "../data"
#filename_train_galaxies_h5     = "training_100gal.hdf5"
filename_train_galaxies_h5     = "test_dc2_training_9816.hdf5"
fullfilename_train_galaxies_h5 = os.path.join(inputdatadir,filename_train_galaxies_h5)

filename_test_galaxies_h5      = "test_dc2_validation_9816.hdf5"
fullfilename_test_galaxies_h5  = os.path.join(inputdatadir,filename_test_galaxies_h5)

In [ ]:
h5file_train = tb.open_file(fullfilename_train_galaxies_h5)
h5file_test = tb.open_file(fullfilename_test_galaxies_h5)

In [ ]:
def convertformat(h5file):
    """
    
    input : H5 file descriptor
    output : datasets
    
    """
    u_mag = np.array(h5file.root.photometry.mag_u_lsst)
    g_mag = np.array(h5file.root.photometry.mag_g_lsst)
    r_mag = np.array(h5file.root.photometry.mag_r_lsst)
    i_mag = np.array(h5file.root.photometry.mag_i_lsst)
    z_mag = np.array(h5file.root.photometry.mag_z_lsst)
    y_mag = np.array(h5file.root.photometry.mag_y_lsst)
    u_err = np.array(h5file.root.photometry.mag_err_u_lsst)
    g_err = np.array(h5file.root.photometry.mag_err_g_lsst)
    r_err = np.array(h5file.root.photometry.mag_err_r_lsst)
    i_err = np.array(h5file.root.photometry.mag_err_i_lsst)
    z_err = np.array(h5file.root.photometry.mag_err_z_lsst)
    y_err = np.array(h5file.root.photometry.mag_err_y_lsst)
    z = np.array(h5file.root.photometry.redshift)
    
    
    #Photometry perturbed: doubling sizes of all errors
    #--------------------------------------------------
    u_magn = u_mag + np.sqrt(1)*u_err*np.random.randn(len(u_mag))
    g_magn = g_mag + np.sqrt(1)*g_err*np.random.randn(len(g_mag))
    r_magn = r_mag + np.sqrt(1)*r_err*np.random.randn(len(r_mag))
    i_magn = i_mag + np.sqrt(1)*i_err*np.random.randn(len(i_mag))
    z_magn = z_mag + np.sqrt(1)*z_err*np.random.randn(len(z_mag))
    y_magn = y_mag + np.sqrt(1)*y_err*np.random.randn(len(y_mag))

    
  
    # First: magnitudes only
    data_mags = np.column_stack((u_mag,g_mag,r_mag,i_mag,z_mag,y_mag))

    # Next: colors only
    data_colors = np.column_stack((u_mag-g_mag, g_mag-r_mag, r_mag-i_mag, i_mag-z_mag, z_mag-y_mag))

    # Next: colors and one magnitude
    data_colmag = np.column_stack((u_mag-g_mag, g_mag-r_mag, r_mag-i_mag, i_mag-z_mag, z_mag-y_mag, i_mag))
    perturbed_colmag=np.column_stack((u_magn-g_magn, g_magn-r_magn, r_magn-i_magn, i_magn-z_magn, z_magn-y_magn, i_magn))

    # Finally: colors, magnitude, and size
    #data_colmagsize = np.column_stack((u_mag-g_mag, g_mag-r_mag, r_mag-i_mag, i_mag-z_mag, z_mag-y_mag, i_mag, rad))

    
    data_z = z
    
    return data_mags, data_colors, data_colmag, perturbed_colmag, data_z
    

In [ ]:
train_data_mags, train_data_colors, train_data_colmag, train_perturbed_colmag, train_z = convertformat(h5file_train)

In [ ]:
test_data_mags, test_data_colors, test_data_colmag, test_perturbed_colmag, test_z = convertformat(h5file_test)

In [ ]:
train_data_mags.shape

In [ ]:
test_data_mags.shape

# A function that we will call a lot: makes the zphot/zspec plot and calculates key statistics

In [ ]:
#A function that we will call a lot: makes the zphot/zspec plot and calculates key statistics
def plot_and_stats(zspec,zphot):
    
    x = np.arange(0,5.4,0.05)
    outlier_upper = x + 0.15*(1+x)
    outlier_lower = x - 0.15*(1+x)

    mask = np.abs((z_phot - z_spec)/(1 + z_spec)) > 0.15
    notmask = ~mask 
    
    # Standard Deviation of the predicted redshifts compared to the data:
    #-----------------------------------------------------------------
    std_result = np.std((z_phot - z_spec)/(1 + z_spec), ddof=1)
    print('Standard Deviation: %6.4f' % std_result)

    # Normalized MAD (Median Absolute Deviation):
    #------------------------------------------
    nmad = 1.48 * np.median(np.abs((z_phot - z_spec)/(1 + z_spec)))
    print('Normalized MAD: %6.4f' % nmad)

    # Percentage of delta-z > 0.15(1+z) outliers:
    #-------------------------------------------
    eta = np.sum(np.abs((z_phot - z_spec)/(1 + z_spec)) > 0.15)/len(z_spec)
    print('Delta z >0.15(1+z) outliers: %6.3f percent' % (100.*eta))
    
    # Median offset (normalized by (1+z); i.e., bias:
    #-----------------------------------------------
    bias = np.median(((z_phot - z_spec)/(1 + z_spec)))
    sigbias=std_result/np.sqrt(0.64*len(z_phot))
    print('Median offset: %6.3f +/- %6.3f' % (bias,sigbias))
    
    # make photo-z/spec-z plot
    #------------------------
    plt.figure(figsize=(8, 8))
    
    #add lines to indicate outliers
    plt.plot(x, outlier_upper, 'k--')
    plt.plot(x, outlier_lower, 'k--')
    plt.plot(z_spec[mask], z_phot[mask], 'r.', markersize=6,  alpha=0.5)
    plt.plot(z_spec[notmask], z_phot[notmask], 'b.',  markersize=6, alpha=0.5)
    plt.plot(x, x, linewidth=1.5, color = 'red')
    plt.title('$\sigma_\mathrm{NMAD} \ = $%6.4f\n'%nmad+'$(\Delta z)>0.15(1+z) $ outliers = %6.3f'%(eta*100)+'%', fontsize=18)
    plt.xlim([0.0, 3])
    plt.ylim([0.0, 3])
    plt.xlabel('$z_{\mathrm{spec}}$', fontsize = 27)
    plt.ylabel('$z_{\mathrm{photo}}$', fontsize = 27)
    plt.grid(alpha = 0.8)
    plt.tick_params(labelsize=15)
    plt.show()
    

In [ ]:
# We need to set up an implementation of the scikit-learn RandomForestRegressor in an object called 'regrn'. 
regrn = RandomForestRegressor(n_estimators = 50, max_depth = 30, max_features = 'auto')

# Relationships of galaxy properties to magnitude

## Plots of colors vs. redshift

In [ ]:
u_mag,g_mag,r_mag,i_mag,z_mag,y_mag = train_data_mags[:,0] , train_data_mags[:,1] , train_data_mags[:,2] , train_data_mags[:,3] , train_data_mags[:,4], train_data_mags[:,5]
z = train_z

In [ ]:
if 1:
# Plots of colors vs. redshift

    fig = plt.figure(figsize=(20, 10))

    
    ax = fig.add_subplot(2,3,1)
    #plt.plot(z, u_mag-g_mag, '.', color = 'red', markersize=3, alpha=0.2)
    ext_range = [ [z.min(),z.max()], [-0.5, 2.5] ]
    ax.hist2d(z, u_mag-g_mag,bins=(50,50),range=ext_range,cmap="Blues")
    # Axis limits:
    #plt.xlim([0., 1.5])
    #plt.ylim([-0.5, 2.5])
    #Axis labels
    ax.set_ylabel('$u-g$ color', fontsize=20)
    ax.set_xlabel('Redshift',fontsize=20)
    #Add grid lines
    ax.grid()


    #plt.figure(figsize=(8, 5))
    ax = fig.add_subplot(2,3,2)

    #plt.plot(z, g_mag-r_mag, '.', color = 'red', markersize=3, alpha=0.2)
    ext_range = [ [z.min(),z.max()], [-0.4, 2.] ]
    ax.hist2d(z, g_mag-r_mag,bins=(50,50),range=ext_range,cmap="Greens")
    #plt.xlim([0., 1.5])
    #plt.ylim([-0.4, 2.])
    ax.set_ylabel('$g-r$ color', fontsize=20)
    ax.set_xlabel('Redshift',fontsize=20)
    ax.grid()


    #plt.figure(figsize=(8, 5))
    ax = fig.add_subplot(2,3,3)
    ext_range = [ [z.min(),z.max()], [-0.3, 1.5] ]
    #plt.plot(z, r_mag-i_mag, '.', color = 'red', markersize=3, alpha=0.2)
    ax.hist2d(z, r_mag-i_mag,bins=(50,50),range=ext_range,cmap="Reds")
    #plt.xlim([0., 1.5])
    #plt.ylim([-0.3, 1.5])
    ax.set_ylabel('$r-i$ color', fontsize=20)
    ax.set_xlabel('Redshift',fontsize=20)
    ax.grid()


    #plt.figure(figsize=(8, 5))
    ax = fig.add_subplot(2,3,4)
    ext_range = [ [z.min(),z.max()], [-0.2, 1.1] ]
    #plt.plot(z, i_mag-z_mag, '.', color = 'red', markersize=3, alpha=0.2)
    ax.hist2d(z, i_mag-z_mag,bins=(50,50),range=ext_range,cmap="Oranges")
    #plt.xlim([0., 1.5])
    #plt.ylim([-0.2, 1.1])
    ax.set_ylabel('$i-z$ color', fontsize=20)
    ax.set_xlabel('Redshift',fontsize=20)
    ax.grid()

    #plt.figure(figsize=(8, 5))
    ax = fig.add_subplot(2,3,5)
    ext_range = [ [z.min(),z.max()], [-0.5, 0.8] ]
    #plt.plot(z, z_mag-y_mag, '.', color = 'red', markersize=3, alpha=0.2)
    ax.hist2d(z, i_mag-z_mag,bins=(50,50),range=ext_range,cmap="Greys")
    #plt.xlim([0., 1.5])
    #plt.ylim([-0.5, 0.8])
    ax.set_ylabel('$z-y$ color', fontsize=20)
    ax.set_xlabel('Redshift',fontsize=20)
    ax.grid()

    #plt.figure(figsize=(8, 5))
    ax = fig.add_subplot(2,3,6)
    ext_range = [ [z.min(),z.max()], [17, 26] ]
    #Plot of i magnitude vs. size
    #plt.plot(z, i_mag, '.', color = 'red', markersize=3, alpha=0.5)
    ax.hist2d(z, i_mag,bins=(50,50),range=ext_range,cmap="GnBu")
    #ax.set_xlim([0., 2.])
    #plt.set_ylim([15.0, 28])
    ax.set_ylabel('$i$ magnitude',fontsize=20)
    ax.set_xlabel('Redshift',fontsize=20)
    ax.grid()




# Running photo-z code and testing the results

## Random forest - 50/50 training/test split

To start, we'll split our data in two randomly, train with 50%, and test with the other 50%.

In [ ]:
if 1:

    # To better assess the quality of the Random Forest fitting, 
    # we split the data into Training (50%) and Test (50%) sets. 
    # The code below performs this task on the data_mags and data_z arrays:
    #data_train, data_test, z_train, z_test = train_test_split(data_mags, data_z, 
    #                    test_size = 0.50, train_size = 0.50, random_state=7182016)
    
    data_train = train_data_mags
    z_train    = train_z
    
    data_test = test_data_mags
    z_test    = test_z  

    #Train the regressor using the training data
    regrn.fit(data_train,z_train)

    #Apply the regressor to predict values for the test data
    z_phot = regrn.predict(data_test)
    z_spec = z_test

    #Make a photo-z/spec-z plot and output summary statistics.
    plot_and_stats(z_spec,z_phot)

## Beware: Do not test with the same objects you use to train an algorithm !!!

Let's try it to see why not...

In [ ]:
if 1:

    # use the regressor trained on the training set, but apply it to the training set instead of the test set
    z_phot = regrn.predict(data_train)
    z_spec = z_train

    plot_and_stats(z_spec,z_phot)

## Note: a better, but slower, way to test: k-fold Cross-validation

In k-fold cross-validation, we split the data into k subsets.  We loop over the subsets, training with all but one and testing with the other; in the end, we get the performance of training with a fraction $k-1 \over k$ of the data, but are able to get test statistics based on the  $entire$ dataset.

This is easy to do in scikit-learn, but does mean running the training k times on more data than before...

In [ ]:
if 1:
    n_folds = 5
    
    data_mags = np.vstack([data_train,data_test])
    data_z = np.hstack([z_train,z_test])

    z_phot = cross_val_predict(regrn, data_mags,data_z, cv=n_folds)
    z_spec = data_z

    plot_and_stats(z_spec,z_phot)

## Now, let's try improving our photo-z's.

First: use colors instead of raw magnitudes.

In [ ]:
if 1:
    #set up the data splits
    #data_train, data_test, z_train, z_test = train_test_split(data_colors, data_z, test_size = 0.50, 
    #                    train_size = 0.50, random_state=7182016)

    
    data_train = train_data_colors
    data_test  = test_data_colors
    
    
    #train the regressor
    regrn.fit(data_train,z_train)

    # run on the test set
    z_phot = regrn.predict(data_test)
    z_spec = z_test

    plot_and_stats(z_spec,z_phot)

__If you're running ahead of the class, try plotting the different magnitudes versus redshift in the code box below, and see how they relate to $z$.__

## Now, let's add one magnitude.

This way, we will include all information we had in the original magnitudes, but in a form that takes more advantage of the regularities in the data.

In [ ]:
if 1:
    #set up the data splits
    #data_train, data_test, z_train, z_test = train_test_split(data_colmag, data_z, test_size = 0.50, 
    #                    train_size = 0.50, random_state=7182016)
    
    
    data_train = train_data_colmag
    data_test = test_data_colmag
    

    #train the regressor
    regrn.fit(data_train,z_train)

    # run on the test set
    z_phot = regrn.predict(data_test)
    z_spec = z_test

    plot_and_stats(z_spec,z_phot)

In [ ]:
r_mag.size

# What happens when training and test sets differ?

## Test 1: Different magnitude ranges

In [ ]:
if 1:
    
    
    
    # split the sample at the median magnitude
    is_bright = r_mag < 23.15

    data_bright = train_data_colmag[is_bright]
    z_bright = train_z[is_bright]
    
    r_mag_test = test_data_mags[:,2]
    is_faint = r_mag_test > 23.15

    data_faint=test_data_colmag[is_faint]
    z_faint = test_z[is_faint]
    

    #train the regressor
    regrn.fit(data_bright,z_bright)

    # run on the test set
    z_phot = regrn.predict(data_faint)
    z_spec = z_faint

    plot_and_stats(z_spec,z_phot)

### while if we run it the opposite way...

In [ ]:
if 1:
    #train the regressor
    regrn.fit(data_faint,z_faint)

    # run on the test set
    z_phot = regrn.predict(data_bright)
    z_spec = z_bright

    plot_and_stats(z_spec,z_phot)

## Test 2: Different colors

In [ ]:
if 1:
    # split the sample based on r-i color
    is_red = r_mag - i_mag > 0.8

    data_red = data_colors[is_red]
    z_red = data_z[is_red]

    data_blue=data_colors[~is_red]
    z_blue = data_z[~is_red]

    #train the regressor
    regrn.fit(data_red,z_red)

    # run on the test set
    z_phot = regrn.predict(data_blue)
    z_spec = z_blue

    plot_and_stats(z_spec,z_phot)

## Test 3: Different redshift ranges

In [ ]:
if 1:
    # split the sample based on redshift
    is_hiz = data_z > 1

    data_hiz = data_colors[is_hiz]
    z_hiz = data_z[is_hiz]

    data_lowz=data_colors[~is_hiz]
    z_lowz = data_z[~is_hiz]

    #train the regressor
    regrn.fit(data_lowz,z_lowz)

    # run on the test set
    z_phot = regrn.predict(data_hiz)
    z_spec = z_hiz

    plot_and_stats(z_spec,z_phot)

   # What effect do magnitude errors have?

In [ ]:
if 1:    
    # let's go back to our original training/test split.
    data_train, data_test, z_train, z_test = train_test_split(data_colmag, data_z, 
                        test_size = 0.50, train_size = 0.50, random_state=7182016)
    
    # we want to use the same splits for the perturbed dataset
    perturbed_train, perturbed_test, pz_train, pz_test = train_test_split(perturbed_colmag, data_z, 
                        test_size = 0.50, train_size = 0.50, random_state=7182016)

In [ ]:
if 1:
    #First, let's see what happens in best case: training set = test set (no errors)
    regrn.fit(data_colmag,data_z)
    z_phot=regrn.predict(data_colmag)
    z_spec=data_z
    plot_and_stats(z_spec,z_phot)

In [ ]:
if 1:
    #Now, let's see when we add errors in this best case
    z_phot=regrn.predict(perturbed_colmag)
    z_spec=data_z
    plot_and_stats(z_spec,z_phot)

In [ ]:
if 1:
    #Now, let's see how much things degrade when we divide training & test properly
    regrn.fit(data_train,z_train)
    z_phot=regrn.predict(perturbed_test)
    z_spec=pz_test
    plot_and_stats(z_spec,z_phot)

__Compare to our original results:__

    Standard Deviation: 0.0747
    Normalized MAD: 0.0220
    Delta z >0.15(1+z) outliers:  4.043 percent
    Median offset: -0.000 +/-  0.001

__Discuss: are our photo-z errors in this catalog dominated by measurement errors, or our training sets?__

# If you have extra time: 

1) Modify `plot_and_stats()` to make plots of ${z_{phot}-z_{zspec}} \over {1 + z_{spec}}$ as a function of redshift and as a function of $i$ magnitude.  Redo all plots.  What trends do you find in the residuals for each?

2) Look at the documentation for scikit-learn regression routines and try one or two other methods.  Can you do better than the random forest results?  There's a code box below you can work with.

3) Modify `plot_and_stats()` to provide the uncertainty in the standard deviation, not just the median shift, to help assess whether improvements are significant. $s (\sigma) = { \sigma \over \sqrt{2N}}$ where $\sigma$ is the standard deviation of the values, $s$ is the uncertainty in that standard deviation, and $N$ is the number of data points.